<font color=blue> 
# Dynamic Programming Tutorial: Policy Iteration
</font> 




### Introduction 

The purpose of this tutorial is to walk-through how to implement Value Iteration, a dynamic programming method. We will use frozenlake, an existing environment developed by openai gym. This tutotial will be broken up into four parts (see below). We will also walk-through the deterministic and stochastic cases and a few discount factors to gain intuition for how this algorithm works.  

### 4 Parts:

- Policy Evaluation (prediction)
- Policy Improvement
- Policy Iteration 
- Value Iteration

Reference: Sutton & Barto, 2018, Reinforcement Learning: An Introduction. 

### Environment 

Reference: https://gym.openai.com/envs/FrozenLake-v0/



__Environment__

A 4x4 gridworld with 4 state types. When you render the environment, the gridworld will be represented as: 

SFFF       
FHFH       
FFFH       
HFFG  

where

- S: starting point, safe
- F: frozen surface, safe
- H: hole, fall to your doom
- G: goal, where the frisbee is located

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

__Actions__
- a == 0 is Left 
- a == 1 is Down
- a == 2 is Right
- a == 3 is Up

In [1]:
import numpy as np
import gym
import time
from lake_envs import *

np.set_printoptions(precision=3)

### Rendering Function -- Do NOT need to modify 

In [2]:
def render_single(env, policy, max_steps=100):
  """
    This function does not need to be modified
    Renders policy once on environment. Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
  """

  episode_reward = 0
  ob = env.reset()
  for t in range(max_steps):
    env.render()
    time.sleep(0.25)
    a = policy[ob]
    ob, rew, done, _ = env.step(a)
    episode_reward += rew
    if done:
      break
  env.render();
  if not done:
    print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
  else:
    print('# Max Steps: ',max_steps)
    print("Episode reward: %f" % episode_reward)

### Examine & Initialize Environments

In [3]:
# Inspect the deterministic environment
env_d = gym.make("Deterministic-4x4-FrozenLake-v0")
print('probability, nextstate, reward, terminal')
env_d.P[0]

probability, nextstate, reward, terminal


{0: [(1.0, 0, 0.0, False)],
 1: [(1.0, 4, 0.0, False)],
 2: [(1.0, 1, 0.0, False)],
 3: [(1.0, 0, 0.0, False)]}

In [4]:
# Inspect the stochastic environment
env_s = gym.make("Stochastic-4x4-FrozenLake-v0")
print('probability, nextstate, reward, terminal')
env_s.P[0]

probability, nextstate, reward, terminal


{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)]}

#### Check for Understanding

Notice that the deterministic environment has 1 cell per parameter, but the stochastic environment has 3 cells. Why is that?

<font color=blue> 
## Policy Evaluation (prediction): 
</font>

The process for evaluating a policy is to successively approximate and update the value of a state using the Bellman equation. The old state's value is replaced with a new value. The new value is obtained by using the old value of the successor state, s', and the rewards we expect to get at the next state. Then, the value function is computer by summing the expectations for all of the successeeding next states.  


### Algorithm: Iterative Policy Evaluation for estimating V~v_pi [p. 75]

Input pi, the policy we are evaluating

Set theta > 0, a small threshold that will determine the accuracy of our policy's estimation

Initialize V(s) arbitrarily, the initial state values, for all states except the terminal state set to zero 

Loop:

    delta <- 0
    Loop for each state in S:
        v <- V(s)
        V(s) <- sum over all actions, pi(a|s) 
                * sum over all next states and their corresponding 
                rewards, p(s',r|s,a)[r + gamma * V(s')]
        delta <- max(delta, |v - V(s)|) 
    until delta < theta


In [5]:
def policy_evaluation(P, nS, nA, policy, gamma=1, tol=1e-3, run_num_episodes=5):
        
    # Initialize value function & delta
    value_function = np.zeros(nS)
    delta = np.inf
    episode = 0
    
    print('Env. Action Probability:',P[0][0])
    print('Initial Value Function',value_function)
    print('Policy',policy)
    
    # Policy eval. will terminate when the value function's change is below the threshold   
    while episode < run_num_episodes:
    # while delta >= tol:    
        
        print('\nEpisode:',episode, ' &  Value Function',value_function)
        
        '''
        prev_value_function = value_function.copy()
        for state in range(nS):
            value_function[state] = 0
            action = policy[state] #policy is deterministic
            for next_tuple in range(len(P[state][action])):
                prob = P[state][action][next_tuple][0]
                nextstate = P[state][action][next_tuple][1]
                reward = P[state][action][next_tuple][2]
                terminal = P[state][action][next_tuple][3]
                #print(reward)
                value_function[state] += prob*(reward + gamma * (1-terminal)*value_function[nextstate])
        delta = np.amax(np.abs(value_function - prev_value_function))
        '''
        
        '''
        # Why do we loop through all (16) states? 
        for s in range(nS):   
            v = value_function[s]
            a = policy[s]
            value_function[s] = 0
            
            for parameter in range(len(P[s][a])):
            
                prob = P[s][a][parameter][0]  
                nextstate = P[s][a][parameter][1]
                reward = P[s][a][parameter][2]
                done = P[s][a][parameter][3]
                
                # Print out P[s][a] as it's useful for debugging.
                print('prob, state, reward, done:',P[s][a])
                #print('value nextstate:',value_function[nextstate])
                
                value_function = prob * (reward + gamma * value_function[nextstate])
                print('value[',s,']:',value_function[s])
                                       
                # Compute the change in value functions across states
                delta = max(delta, np.abs(v - value_function[s]))
        '''
        
        episode += 1
        
    # Final value function
    print('Final # of Episodes: ',episode)
    
    return value_function

## <font color=blue>Evaluate Deterministic Policies</font>

### Examine a Deterministic Policies for a Discount Value of 1

In [6]:
# Initialize a Deterministic Zeros Policy
policy = np.zeros(env_d.nS, dtype=int)
print('Policy: ',policy)

Policy:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
# Evaluate a Deterministic Zeros Policy 
print("\n" + "-"*31 + "\nBeginning Zero Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, policy, gamma=1, tol=1e-3, run_num_episodes=3)


-------------------------------
Beginning Zero Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  3


In [8]:
# Examine a Deterministic Zeros Policy & Values
print('Policy: ',policy)
print('Values: ',state_values)

Policy:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Check for Understanding -- Zero Policy

Does this value function make sense to you? Why or why not?

In [9]:
# Inspect Behavior for a Deterministic Zeros Policy
render_single(env_d, policy, max_steps=3)


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
The agent didn't reach a terminal state in 3 steps.


#### Check for Understanding -- Zero Policy

Our agent doesn't reach a terminal state in 5 steps. It turns out that if we instead tried for 100 steps, our agent still wouldn't reach a terminal state. In an environment of only 16 states and 64 actions, it seems like we should reach a terminal states, so why don't we? (Hint: Observe the behavior of the highlighted state.)

If the values of our value function didn't make sense to you before, do they now? What new realization did you have?

In [10]:
# Initialize a Ones Policy
policy = np.ones(env_d.nS, dtype=int)

# Evaluate a Ones Policy 
print("\n" + "-"*31 + "\nBeginning Ones Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, policy, gamma=1, tol=1e-3, run_num_episodes=3)

# Examine a Ones Policy & Values
print('\nPolicy: ',policy)
print('Final Values: ',state_values)

# Inspect the Behavior of a Ones Policy
render_single(env_d, policy, max_steps=3)


-------------------------------
Beginning Ones Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  3

Policy:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Final Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
# Max Steps:  3
Episode reward: 0.000000


#### Check for Understanding -- Ones Policy

This policy does reach a terminal state after only 5 steps. Additionally, there is a difference in behavior with 
a policy of ones. What is that behavior? What does this mean? (Hint: If you're unsure test for policies of all twos, threes, fours and so on until you've figured it out.)

For a twos policy, the agent would move to the right and get stuck after reaching state 3, while in the case of a threes policy it would get stuck in trying to move up and thus would never leave the starting state. This means that you will not see any value changes. 

### Examine Deterministic Policies for a  <font color=blue>Discount Value of 0.9</font>



In [11]:
# Initialize a Deterministic Zeros Policy
policy = np.zeros(env_d.nS, dtype=int)

# Evaluate a Deterministic Zeros Policy 
print("\n" + "-"*31 + "\nBeginning Zero Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, policy, gamma=0.9, tol=1e-3, run_num_episodes=3)


-------------------------------
Beginning Zero Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  3


In [12]:
# Examine a Deterministic Zeros Policy & Values
print('\nPolicy: ',policy)
print('Final Values: ',state_values)

# Inspect Behavior for a Deterministic Zeros Policy
render_single(env_d, policy, max_steps=3)


Policy:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Final Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
The agent didn't reach a terminal state in 3 steps.


In [13]:
# Initialize a Deterministic Twos Policy
policy = np.array([2]*env_d.nS, dtype=int)

# Evaluate a Determinstic Twos Policy 
print("\n" + "-"*31 + "\nBeginning Twos Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, policy, gamma=0.9, tol=1e-3)


-------------------------------
Beginning Twos Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5


In [14]:
# Examine a Deterministic Twos Policy & Values
print('\nPolicy: ',policy)
print('Values: ',state_values)

# Inspect Behavior for a Deterministic Twos Policy
render_single(env_d, policy, max_steps=5)


Policy:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
The agent didn't reach a terminal state in 5 steps.


### Check for Understanding 

Why don't we reach a terminal state in this case?

For all of these policies we have the same issue, the same action at every state doesn't lead the agent to the goal state. So what can we do? Let's be random!

### Evaluate a Deterministic  <font color=blue>Random </font> Policy for Discount Factor 0.9

In [15]:
rpolicy = np.random.choice(env_d.nA, env_d.nS)
print('\nPolicy: ',rpolicy)


Policy:  [2 1 2 3 2 2 3 1 3 1 2 1 0 2 1 3]


Although randomness might increase our chances of getting somewhere closer to the goal state, it still doesn't produce the goal driven behavior. Knowing that moving into the goal state provide the agent with reward, let's add an action that will produce reward. 

In [17]:
# Evaluate a Deterministic Random Policy 
print("\n" + "-"*31 + "\nBeginning Random Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, rpolicy, gamma=0.9, tol=1e-3, run_num_episodes=5)

# Examine a Deterministic Random Policy & Values
print('\nPolicy: ',rpolicy)
print('Values: ',state_values)


-------------------------------
Beginning Random Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [2 1 2 3 2 2 3 1 3 1 2 1 0 2 1 3]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5

Policy:  [2 1 2 3 2 2 3 1 3 1 2 1 0 2 1 3]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
# Force the agent to move to the right 
rpolicy[14] = 2

In [19]:
# Evaluate a Deterministic Random Policy 
print("\n" + "-"*31 + "\nBeginning Random Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, rpolicy, gamma=0.9, tol=1e-3, run_num_episodes=3)

# Examine a Deterministic Random Policy & Values
print('\nPolicy: ',rpolicy)
print('Values: ',state_values)


-------------------------------
Beginning Random Policy Iteration
-------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [2 1 2 3 2 2 3 1 3 1 2 1 0 2 2 3]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  3

Policy:  [2 1 2 3 2 2 3 1 3 1 2 1 0 2 2 3]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
# Inspect Behavior for a Deterministic Twos Policy
render_single(env_d, rpolicy, max_steps=10)


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
# Max Steps:  10
Episode reward: 0.000000


#### Check for Understanding 

Does rigging the policy impact our value function?

## <font color=blue>Evaluate Stochastic Policies </font>

### Examine Stochastic Policies for Discount Value of 1

In [21]:
# Recall we initialized a stochastic environment
env_s = gym.make("Stochastic-4x4-FrozenLake-v0")

### Check for Understanding

As before, we could evaluate and examine what happens for a zero, one ...etc. stochastic policies, but you should be able to say. What is that behavior? 

In [22]:
# Initialize a Stochastic Twos Policy
policy = np.array([2]*env_s.nS, dtype=int)

# Evaluate a Stochastic Twos Policy 
print("\n" + "-"*31 + "\nBeginning Twos Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_s.P, env_s.nS, env_s.nA, policy, gamma=1, tol=1e-3)



-------------------------------
Beginning Twos Policy Iteration
-------------------------------
Env. Action Probability: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5


In [23]:
# Examine a Stochastic Twos Policy & Values
print('\nPolicy: ',policy)
print('Values: ',state_values)

# Inspect Behavior for a Stochastic Twos Policy
render_single(env_s, policy, max_steps=10)


Policy:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
# Max Steps:  10
Episode reward: 0.000000


### Examine Stochastic Policies for Discount Value of 0.9

In [24]:
# Initialize a Stochastic Twos Policy
policy = np.array([2]*env_s.nS, dtype=int)

# Evaluate a Stochastic Twos Policy 
print("\n" + "-"*31 + "\nBeginning Twos Policy Iteration\n" + "-"*31)
state_values = policy_evaluation(env_s.P, env_s.nS, env_s.nA, policy, gamma=0.9, tol=1e-3)

# Examine a Stochastic Twos Policy & Values
print('\nPolicy: ',policy)
print('Values: ',state_values)


-------------------------------
Beginning Twos Policy Iteration
-------------------------------
Env. Action Probability: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5

Policy:  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [25]:
# Inspect Behavior for a Stochastic Twos Policy
render_single(env_s, policy, max_steps=3)


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
The agent didn't reach a terminal state in 3 steps.


### Examine Stochastic Random Policies for Discount Value 0.9

In [26]:
# Initialize Policy 
rpolicy = np.random.choice(env_d.nA, env_d.nS)

# Evaluate a Deterministic Random Policy 
print("\n" + "-"*45 + "\nBeginning Stochastic Random Policy Iteration\n" + "-"*45)
state_values = policy_evaluation(env_d.P, env_d.nS, env_d.nA, rpolicy, gamma=0.9, tol=1e-3, run_num_episodes=5)


---------------------------------------------
Beginning Stochastic Random Policy Iteration
---------------------------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [0 0 0 3 0 2 2 1 0 0 2 3 3 1 3 2]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5


In [27]:
# Examine a Deterministic Random Policy & Values
print('\nPolicy: ',rpolicy)
print('Values: ',state_values)

# Inspect Behavior for a Stochastic Twos Policy
render_single(env_s, rpolicy, max_steps=3)


Policy:  [0 0 0 3 0 2 2 1 0 0 2 3 3 1 3 2]
Values:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
The agent didn't reach a terminal state in 3 steps.


<font color=blue> 
# Policy Iteration 
</font>

Policy Iteration (using iterative policy evaluation) for estimating pi ~= pi*

1. Initialization 
V(s) is an element of the real numbers for all states
pi(s) is an element of A(s) for all states 

2. Policy Evaluation (implemented above)
Loop: 
    delta <- 0
    Loop for each s in the set of States:
        v <- V(s)
        V(s) <- sum over all next states & rewards p(s',r|s,a) * [r + gamma * V(s')]
        delta <- max(delta, |v-V(s)|)
    until delta < theta 
    
    
3. Policy Improvement (still need to implement)

policy-stable <- true

For each s in the set of States:

    old-action <- pi(s)   
    pi(s) <- argmax over actions sum over all next states and rewards, p(s',r|s,a)[r + gamma * V(s)] 
    If old-action =/= pi(s), then policy-stable <- false 
    
If policy-stable, then stop and return V~=~v* & pi~=~pi*; else go to 2
        

In [28]:
def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=1):
   
    new_policy = np.zeros(nS, dtype='int')

    action_values = np.zeros(nA)
    for s in range(nS):
        for a in range(nA):
            for parameter in range(len(P[s][a])):
            
                prob = P[s][a][parameter][0]  
                nextstate = P[s][a][parameter][1]
                reward = P[s][a][parameter][2]
                done = P[s][a][parameter][3]
                
            print('prob, state, reward, done:',P[s][a])
            #print('value nextstate:',value_function[nextstate])
            
            action_values[a] += prob * (reward + (1-done) * gamma * value_from_policy[nextstate])
            
        new_policy[s] = np.argmax(action_values)       
        
    return new_policy

In [28]:
def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3):
    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)

    loop = 0 
    policy_stable = False
    #while not policy_stable:
    while loop < 10:
        
        # Policy Evaluation
        value_from_policy = policy_evaluation(P, nS, nA, policy, gamma=gamma, tol=tol)
        
        # Policy Improvement
        new_policy = policy_improvement(P, nS, nA, value_from_policy, policy, gamma=gamma)
        
        loop += 1 
        print('Loop:',loop)
        
        if (policy == new_policy).all():
            policy_stable = True
            
        policy = new_policy.copy()
        value_function = value_from_policy.copy()

    return value_function, policy

### Evaluate & Improve a <font color=blue>Deterministic</font> Random Policy with Discount Factor 0.9

In [29]:
# Initialize a random policy 
policy = np.random.choice(env_d.nA, env_d.nS)

# Evaluate your policy's value function
value_from_policy = policy_evaluation(env_d.P, env_d.nS, env_d.nA, policy, gamma=0.9, tol=1e-3, run_num_episodes=10)

Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [3 3 1 0 3 1 3 0 2 3 2 2 1 3 3 3]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 5  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 6  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 7  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 8  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 9  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  10


In [30]:
print('Deterministic Policy:',policy)
print('Value Function:', value_from_policy)

Deterministic Policy: [3 3 1 0 3 1 3 0 2 3 2 2 1 3 3 3]
Value Function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [31]:
policy_improvement(env_d.P, env_d.nS, env_d.nA, state_values, policy)

prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 4, 0.0, False)]
prob, state, reward, done: [(1.0, 1, 0.0, False)]
prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 5, 0.0, True)]
prob, state, reward, done: [(1.0, 2, 0.0, False)]
prob, state, reward, done: [(1.0, 1, 0.0, False)]
prob, state, reward, done: [(1.0, 1, 0.0, False)]
prob, state, reward, done: [(1.0, 6, 0.0, False)]
prob, state, reward, done: [(1.0, 3, 0.0, False)]
prob, state, reward, done: [(1.0, 2, 0.0, False)]
prob, state, reward, done: [(1.0, 2, 0.0, False)]
prob, state, reward, done: [(1.0, 7, 0.0, True)]
prob, state, reward, done: [(1.0, 3, 0.0, False)]
prob, state, reward, done: [(1.0, 3, 0.0, False)]
prob, state, reward, done: [(1.0, 4, 0.0, False)]
prob, state, reward, done: [(1.0, 8, 0.0, False)]
prob, state, reward, done: [(1.0, 5, 0.0, True)]
prob, state, reward, done: [(1.0, 0, 0.0, False)]
pro

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2])

In [32]:
print("\n" + "-"*25 + "\nBeginning Policy Iteration\n" + "-"*25)

# Evaluate & Improve Deterministic Random Policy
V_pi, p_pi = policy_iteration(env_d.P, env_d.nS, env_d.nA, gamma=0.9, tol=1e-3)



-------------------------
Beginning Policy Iteration
-------------------------
Env. Action Probability: [(1.0, 0, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5
prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 4, 0.0, False)]
prob, state, reward, done: [(1.0, 1, 0.0, False)]
prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 0, 0.0, False)]
prob, state, reward, done: [(1.0, 5, 0.0, True)]
prob, state, reward, done: [(1

In [33]:
# Inspect Behavior 
render_single(env_d, p_pi, 100)


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFF

### Evaluate and Improve a <font color=blue> Stochastic</font> Random Policy with Discount Factor 0.9



In [38]:
# Initialize a stochastic random policy 
policy = np.random.choice(env_s.nA, env_s.nS)
policy[14] = 2
print("\n" + "-"*25 + "\nBeginning Policy Iteration\n" + "-"*25)

# Evaluate & Improve Deterministic Random Policy
V_pi, p_pi = policy_iteration(env_s.P, env_s.nS, env_s.nA, gamma=0.9, tol=1e-3)


-------------------------
Beginning Policy Iteration
-------------------------
Env. Action Probability: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]
Initial Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Policy [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Episode: 0  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 1  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 2  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 3  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Episode: 4  &  Value Function [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Final # of Episodes:  5
prob, state, reward, done: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]
prob, state, reward, done: [(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False),